In [1]:
import os
import tensorflow as tf
import numpy as np
from numpy import sqrt, float32, array
import pandas as pd
import pickle
import cv2
import tensorflow.python.platform
from keras.preprocessing import sequence
from collections import Counter

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# !pip install --upgrade pip

In [3]:
# ! pip list --uptodate

In [4]:
# !pip install opencv-python

In [5]:
model_path = '../save_model_final'
model_path_transfer = '../save_model_final'
# feature_path = '../data/feats.npy'
# annotation_path_kr = '../data/korean_data/ko_translateGoogle_caption.txt'
feature_path = '../test0607.txt' #벡터화된 키워드
annotation_path_kr = '../para0607.txt'

### Loading data

In [6]:
def get_data(annotation_path_kr, feature_path):
    with open('para0607.txt', 'rb') as f:
        captions = np.array(pickle.load(f))
        # captions = pickle.load(f)
        # captions = captions[0:100]
    with open('test0607.txt', 'rb') as f:
        feats = np.array(pickle.load(f))
        # feats = pickle.load(f)
        # feats = feats[0:100]
        
    return feats, captions

In [7]:
# pickle에 저장하기
# with open('sentence.txt', 'wb') as f:
#     pickle.dump(sentence, f)
# f.close

In [5]:
# data load
feats, captions = get_data(annotation_path_kr, feature_path)

In [6]:
# 확인
print(feats.shape) #임베딩 백터
print(captions.shape) #문장
print(captions[1])
print(feats[1])

(12360, 2000)
(12360,)
평소 CM에 관심이 있던 후배, 동기와 함께 팀을 이루어 공모전에 참여했습니다. 총 6명이 한 조를 이루었고 저는 리더를 맡았습니다. 상황이 좋지 않았습니다. 부지선정부터 공법선정 등 준비해야 할 범위가 너무 컸으며 더불어 시험과 과제제출 등이 겹치면서 준비에 어려움이 있었습니다. 저는 리더로서 두 가지 방법을 제안하여 문제를 해결하고자 했습니다. 흥미에 맞는 분야를 담당: 열정적으로 과업을 수행하도록 했습니다. 피드백 모임- 개인이 미처 보지 못한 부분을 점검하도록 했습니다. 그 외에도 팀원끼리 지속해서 연락하여 서로 모르는 부분을 채워가며 공모전을 준비했습니다. 그 결과 바쁜 일정 와중에도 모두가 만족하는 작품을 완성할 수 있었습니다. 비록, 순위에는 들지 못했지만, 사업 전반을 계획해보고 공사의 흐름을 파악할 수 있었던 값진 경험이었습니다. 이 경험을 통해 구성원 간 소통의 중요성과 협업을 통해 난관을 이겨내는 법을 배웠습니다. 다수의 업체와 협력이 이루어지는 건설 현장에서 소통을 통해 갈등을 유연하게 해결하고 협력을 통해 효율적으로 목표를 달성하겠습니다.
[ 0.724064   -0.5830169   0.00841407 ... -0.24805881  0.2740903
 -0.49066275]


In [8]:
def preProBuildWordVocab(sentence_iterator, word_count_threshold=20): # function from Andre Karpathy's NeuralTalk
    print('preprocessing %d word vocab' % (word_count_threshold, ))
    word_counts = {}
    nsents = 0
    for sent in sentence_iterator:
        nsents += 1
        for w in sent.lower().split(' '):
            word_counts[w] = word_counts.get(w, 0) + 1
    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '.'  
    wordtoix = {}
    wordtoix['#START#'] = 0 
    ix = 1
    for w in vocab:
        wordtoix[w] = ix
        ixtoword[ix] = w
        ix += 1

    word_counts['.'] = nsents
    bias_init_vector = np.array([1.0*word_counts[ixtoword[i]] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector) 
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector)
    
    return wordtoix, ixtoword, bias_init_vector.astype(np.float32)

In [9]:
class Caption_Generator():
    def __init__(self, dim_in, dim_embed, dim_hidden, batch_size, n_lstm_steps, n_words, init_b):
        
        self.dim_in = dim_in
        self.dim_embed = dim_embed
        self.dim_hidden = dim_hidden
        self.batch_size = batch_size
        self.n_lstm_steps = n_lstm_steps
        self.n_words = n_words
        
        # declare the variables to be used for our word embeddings
        with tf.device("/cpu:0"): # tf.device("/cpu:0"):
            self.word_embedding = tf.Variable(tf.random_uniform([self.n_words, self.dim_embed], -0.1, 0.1), name='word_embedding')
            
        self.embedding_bias = tf.Variable(tf.zeros([self.dim_embed]), name='embedding_bias')
        
        # declare the LSTM itself
        # self.lstm = tf.contrib.rnn.BasicLSTMCell(dim_hidden, state_is_tuple=True)
        
        
        ########## multi-layer ##########
        def lstm_cell():
            cell = tf.contrib.rnn.BasicLSTMCell(dim_hidden, state_is_tuple=True)
            return cell

        self.lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell() for i in range(3)], state_is_tuple=True)
        ################################
        
        
        # declare the variables to be used to embed the image feature embedding to the word embedding space
        self.img_embedding = tf.Variable(tf.random_uniform([dim_in, dim_hidden], -0.1, 0.1), name='img_embedding')
        self.img_embedding_bias = tf.Variable(tf.zeros([dim_hidden]), name='img_embedding_bias')

        # declare the variables to go from an LSTM output to a word encoding output
        self.word_encoding = tf.Variable(tf.random_uniform([dim_hidden, n_words], -0.1, 0.1), name='word_encoding')
        
        # initialize this bias variable from the preProBuildWordVocab output
        self.word_encoding_bias = tf.Variable(init_b, name='word_encoding_bias')

    def build_model(self):
        # declaring the placeholders for our extracted image feature vectors, our caption, and our mask
        # (describes how long our caption is with an array of 0/1 values of length `maxlen`  
        img = tf.placeholder(tf.float32, [self.batch_size, self.dim_in])
        caption_placeholder = tf.placeholder(tf.int32, [self.batch_size, self.n_lstm_steps])
        mask = tf.placeholder(tf.float32, [self.batch_size, self.n_lstm_steps])
                
        # getting an initial LSTM embedding from our image_imbedding
        image_embedding = tf.matmul(img, self.img_embedding) + self.img_embedding_bias
        # setting initial state of our LSTM
        state = self.lstm.zero_state(self.batch_size, dtype=tf.float32)

        total_loss = 0.0
        with tf.variable_scope("RNN"):
            for i in range(self.n_lstm_steps): 
                if i > 0:
                   #if this isn’t the first iteration of our LSTM we need to get the word_embedding corresponding
                   # to the (i-1)th word in our caption 
                    with tf.device("/cpu:0"): # tf.device("/cpu:0"):
                        current_embedding = tf.nn.embedding_lookup(self.word_embedding, caption_placeholder[:,i-1]) + self.embedding_bias
                else:
                     #if this is the first iteration of our LSTM we utilize the embedded image as our input 
                    current_embedding = image_embedding
                if i > 0: 
                    # allows us to reuse the LSTM tensor variable on each iteration
                    tf.get_variable_scope().reuse_variables()
                
                current_embedding = tf.expand_dims(current_embedding, 1)
                out, state = tf.nn.dynamic_rnn(cell=self.lstm, inputs=current_embedding
                                                               ,initial_state=state)

                if i > 0:
                    #get the one-hot representation of the next word in our caption 
                    labels = tf.expand_dims(caption_placeholder[:, i], 1)
                    ix_range = tf.range(0, self.batch_size, 1)
                    ixs = tf.expand_dims(ix_range, 1)
                    concat = tf.concat([ixs, labels],1)
                    onehot = tf.sparse_to_dense(
                            concat, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)

                    #perform a softmax classification to generate the next word in the caption
                    out = tf.squeeze(out, [1])
                    logit = tf.matmul(out, self.word_encoding) + self.word_encoding_bias
                    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=onehot)
                    xentropy = xentropy * mask[:,i]

                    loss = tf.reduce_sum(xentropy)
                    total_loss += loss

            total_loss = total_loss / tf.reduce_sum(mask[:,1:])
            return total_loss, img,  caption_placeholder, mask

In [10]:
### Parameters ###
dim_embed = 50 # dimension of word embedding
dim_hidden = 50 # dimension of all hidden state size
dim_in = 2000 # input dimension
batch_size = 30
momentum = 0.9
n_epochs = 150

def train(learning_rate=0.001, continue_training=False, transfer=True):
    
    tf.reset_default_graph()

    feats, captions = get_data(annotation_path_kr, feature_path)
    wordtoix, ixtoword, init_b = preProBuildWordVocab(captions)

    np.save('../DJ/data/ixtoword_kr', ixtoword)

    index = (np.arange(len(feats)).astype(int))
    np.random.shuffle(index)


    sess = tf.InteractiveSession()
    n_words = len(wordtoix)
    maxlen = np.max( [x for x in map(lambda x: len(x.split(' ')), captions)] )
    
    caption_generator = Caption_Generator(dim_in, dim_hidden, dim_embed, batch_size, maxlen+2, n_words, init_b)

    
    loss, image, sentence, mask = caption_generator.build_model()

    saver = tf.train.Saver(max_to_keep=100)
    global_step=tf.Variable(0,trainable=False)
    learning_rate = tf.train.exponential_decay(learning_rate, global_step, int(len(index)/batch_size), 0.95)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    tf.global_variables_initializer().run()

    if continue_training:
        if not transfer:
            saver.restore(sess,tf.train.latest_checkpoint(model_path))
        else:
            saver.restore(sess,tf.train.latest_checkpoint(model_path_transfer))
            
    losses=[]
    for epoch in range(n_epochs):
        for start, end in zip( range(0, len(index), batch_size), range(batch_size, len(index), batch_size)):

            current_feats = feats[index[start:end]]
            current_captions = captions[index[start:end]]
            current_caption_ind = [x for x in map(lambda cap: [wordtoix[word] for word in cap.lower().split(' ')[:-1] if word in wordtoix], current_captions)]

            current_caption_matrix = sequence.pad_sequences(current_caption_ind, padding='post', maxlen=maxlen+1)
            current_caption_matrix = np.hstack( [np.full( (len(current_caption_matrix),1), 0), current_caption_matrix] )

            current_mask_matrix = np.zeros((current_caption_matrix.shape[0], current_caption_matrix.shape[1]))
            nonzeros = np.array([x for x in map(lambda x: (x != 0).sum()+2, current_caption_matrix )])

            for ind, row in enumerate(current_mask_matrix):
                row[:nonzeros[ind]] = 1

            _, loss_value = sess.run([train_op, loss], feed_dict={
                image: current_feats.astype(np.float32),
                sentence : current_caption_matrix.astype(np.int32),
                mask : current_mask_matrix.astype(np.float32)
                })

            print("Current Cost: ", loss_value, "\t Epoch {}/{}".format(epoch, n_epochs), "\t Iter {}/{}".format(start,len(feats)))
        print("Saving the model from epoch: ", epoch)
        saver.save(sess, os.path.join(model_path, 'model'), global_step=epoch)
        losses.append(loss_value)
    
    # pickle에 저장하기
    with open('loss_0607.txt', 'wb') as f:
        pickle.dump(losses, f)
    f.close

# Train

In [ ]:
try:
    train(.001,False,False)    #train from scratch
    #train(.001,True,True)    #continue training from pretrained weights @epoch500
    #train(.001,True,False)  #train from previously saved weights
except KeyboardInterrupt:
    print('Exiting Training')

preprocessing 20 word vocab
preprocessed words 208030 -> 10370


In [17]:
with open('loss_0607.txt', 'rb') as f:
        loss = pickle.load(f)

In [19]:
loss

[5.998882, 5.9972706]